In [1]:
# IMPORTING MODULES
import numpy as np
import tensorflow as tf
import scipy.io as sci
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
import numpy as np
from Datasets import *
from CelebAGenerator import *
from SVHNGenerator import *
K.set_learning_phase(0)
from glob import glob
import os
import numpy as np
from tqdm import tqdm
from skimage.measure import compare_psnr, compare_ssim
import scipy.misc as scipy

Using TensorFlow backend.


In [7]:
# LOADING VARIABLES
var1 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.01 subsamp')
spacing= var1['spacing'].astype('float32')
P_op = var1['P_op'].astype('float32')
samplingIndices = var1['samplingIndices'].astype('int32')
pupil = var1['pupil'].astype('float32')[:,:,1]
h = 56#var1['h'].astype('float32')
w = 56#var1['w'].astype('float32')

###############################################################################
###################### this portion is for subsampling experiments only #######
###############################################################################
#var1 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.01 subsamp')
#var2 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.02 subsamp')
#var3 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.03 subsamp')
#var4 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.05 subsamp')
#var5 = sci.loadmat('Parameters for 56 image,15 Dia,0.65 overlap,9 grid,0.10 subsamp')
#spacing= var1['spacing'].astype('float32')

#P_op1 = var1['P_op'].astype('float32')
#P_op2 = var2['P_op'].astype('float32')
#P_op3 = var3['P_op'].astype('float32')
#P_op4 = var4['P_op'].astype('float32')
#P_op5 = var5['P_op'].astype('float32')


#samplingIndices = var1['samplingIndices'].astype('int32')
#pupil = var1['pupil'].astype('float32')[:,:,1]
#h = 56#var1['h'].astype('float32')
#w = 56#var1['w'].astype('float32')


###############################################################################
###############################################################################
###############################################################################

# HYPERPARAMETERS
LEARNING_RATE =  0.05   #0.001
RANDOM_RESTARTS = 1
#NOISE_STD       = 0.005
STEPS           = 1000
optimizer       = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)

PLOT_LOSS       = True
SAVE_RESULTS    = True
rand_rest       = 1 #random restarts
latent_dim      = 50
#L               = 64
cam_arr         = P_op.shape[2]

# FUNCTION DEFINITIONS
def fftshift(x):
    xx = tf.reshape(x,[L,L,-1])
    a =xx[0:int(L/2),0:int(L/2),:]
    b =xx[0:int(L/2),int(L/2):L,:]
    c =xx[int(L/2):L,0:int(L/2),:]
    d =xx[int(L/2):L,int(L/2):L,:]
    B1 =tf.concat([d,c],axis=1)
    B2 =tf.concat([b,a],axis=1)
    B3 =tf.concat([B1,B2],axis=0)
    B3 =tf.reshape(B3,[L,L,-1])
    return B3


def forward_map(x): # change this for subsampling remove P_op
    h = tf.constant(L);xx=tf.to_complex64(tf.reshape(x,[L,L]));
    X   = tf.reshape(fftshift(tf.fft2d(xx)),[h,h])
    X1   = tf.reshape(tf.pad(X, tf.constant([[int(np.floor(spacing*(9-1)/2)),int(np.floor(spacing*(9-1)/2))],[int(np.floor(spacing*(9-1)/2)),int(np.floor(spacing*(9-1)/2))]]), "CONSTANT"),[-1,1])
    ind1 = tf.reshape(tf.gather(X1,tf.constant(samplingIndices-1)),[h,h,-1])
    ifft_mul = tf.ifft2d(tf.to_complex64(tf.multiply(tf.transpose(ind1),tf.constant(pupil,dtype=tf.complex64))))
    abs_ifft_mul = tf.transpose((tf.abs(ifft_mul)),[1,2,0])
    sub_samp = tf.multiply(tf.constant(P_op),abs_ifft_mul)
    return [sub_samp]

# LOADING CelebA GENERATOR
#gen = CelebAGenerator()
#gen.GenerateModel()
#gen.LoadWeights()
#G = gen.GetModels()
#G.trainable = False

DATASET         = "mnist"                  # "celeba" or "mnist"
# LOADING GENERATOR
if DATASET == "celeba":
    gen = CelebAGenerator()
    gen.GenerateModel()
    gen.LoadWeights()
    G = gen.GetModels()
    L  = 64
    
if DATASET == "mnist":
    gen = SVHNGenerator()
    gen.GenerateModel()
    gen.LoadWeights()
    _, _, G = gen.GetModels()
    L   = 56
#G.trainable = False

In [3]:
pupil.shape

(56, 56)

$\Huge{\textbf{ Range subsample Script (in loop)} (MNIST)}$

In [ ]:
Orig_Path   = './original_images/mnist/*.png'
Orig_Path   = './original_images/mnist/*.png'
P_op_arr = [P_op1,P_op2,P_op3,P_op4,P_op5]


for kk in range(len(X_Orig)):
    images = []
    im1 = X_Orig[kk]
    
    for j in range(len(P_op_arr)):
        

        P_op = P_op_arr[j]
        
        
        if DATASET == "celeba":
            gen = CelebAGenerator()
            gen.GenerateModel()
            gen.LoadWeights()
            G = gen.GetModels()
            L  = 64
    
        if DATASET == "mnist":
            gen = SVHNGenerator()
            gen.GenerateModel()
            gen.LoadWeights()
            _, _, G = gen.GetModels()
            L   = 56
        G.trainable = False

        im = im1 
        abs_ifft_mul = []


        v1 = im
        #v = (v1/255).astype('float32')
        k = (samplingIndices-1);
        X   = np.fft.fftshift(np.fft.fft2(v1))#/(h*w)**0.5 
        Xx   = np.reshape(np.pad(X,int(np.floor(spacing*(9-1))/2),'constant'),[-1,1])
        ind = np.reshape(Xx[k],[L,L,-1])
        ifft_mul = np.fft.ifft2(ind.transpose()*pupil)
        ifft_mul = np.transpose(ifft_mul,[1,2,0])
        abs_ifft_mul = (P_op*np.abs(ifft_mul))

        #abs_ifft_mul = np.clip((abs_ifft_mul + np.random.normal(0,0,size=abs_ifft_mul.shape)),0,1)

        #plt.subplot(1,2,1)
        #plt.imshow(im)
        #plt.subplot(1,2,2)
        #plt.imshow(abs_ifft_mul_display[np.int(cam_arr/2),:,:,:])
        # FORWARD MAP
        z_tf                = tf.Variable(tf.random_normal(shape=(rand_rest, latent_dim)))
        Y_tf                = tf.placeholder(dtype="float32", shape=(L,L,cam_arr)) #PUT Y HERE AND CHANGE SHAPE

        xG_tf               = G(z_tf)[:,:,:,0]
        #xG_tf               = (xG_tf+1)/2
        yGpred_tf           = forward_map(xG_tf,P_op)
        Loss_tf             = tf.reduce_mean( tf.square( Y_tf - yGpred_tf ))
        opt                 = optimizer.minimize(Loss_tf, var_list=[z_tf])

        # SOLVING LEAST SQUARES
        # SOLVING LEAST SQUARES
        sess = K.get_session()
        sess.run(tf.variables_initializer([z_tf]))
        Loss = []
        for i in tqdm(range(STEPS)):
            _ , loss = sess.run([opt,Loss_tf], feed_dict={Y_tf:abs_ifft_mul}) #put Y and A here too
            if (i %50) == 0:
                x_hat= sess.run([xG_tf])
                x_hat = np.array(x_hat)
                print(compare_ssim(im1,x_hat[0,0,:,:].astype('float64')))
                plt.imshow(x_hat[0,0,:,:],cmap='gray')
                plt.show() 
            Loss.append(np.mean(loss))
        
        scipy.imsave('test'+str(kk+1)+'_subsampled_'+str(j+1)+'.png',x_hat[0,0,:,:])

$\Huge{\textbf{ DeepPtych+ code for MNIST -- Subsampling}}$

In [ ]:
Orig_Path   = './original_images/mnist/*.png'
X_Orig = np.array([ imread(path) for path in glob(Orig_Path)])/255
P_op_arr = [P_op1,P_op2,P_op3,P_op4,P_op5]


for kk in range(len(X_Orig)):
    images = []
    im1 = X_Orig[kk]
    
    for j in range(len(P_op_arr)):
        

        P_op = P_op_arr[j]
        
        
        if DATASET == "celeba":
            gen = CelebAGenerator()
            gen.GenerateModel()
            gen.LoadWeights()
            G = gen.GetModels()
            L  = 64
    
        if DATASET == "mnist":
            gen = SVHNGenerator()
            gen.GenerateModel()
            gen.LoadWeights()
            _, _, G = gen.GetModels()
            L   = 56
        G.trainable = False

        im = im1 
        abs_ifft_mul = []


        v1 = im
        #v = (v1/255).astype('float32')
        k = (samplingIndices-1);
        X   = np.fft.fftshift(np.fft.fft2(v1))#/(h*w)**0.5 
        Xx   = np.reshape(np.pad(X,int(np.floor(spacing*(9-1))/2),'constant'),[-1,1])
        ind = np.reshape(Xx[k],[L,L,-1])
        ifft_mul = np.fft.ifft2(ind.transpose()*pupil)
        ifft_mul = np.transpose(ifft_mul,[1,2,0])
        abs_ifft_mul = (P_op*np.abs(ifft_mul))

        #abs_ifft_mul = np.clip((abs_ifft_mul + np.random.normal(0,0,size=abs_ifft_mul.shape)),0,1)

        #plt.subplot(1,2,1)
        #plt.imshow(im)
        #plt.subplot(1,2,2)
        #plt.imshow(abs_ifft_mul_display[np.int(cam_arr/2),:,:,:])
        # FORWARD MAP
        # FORWARD MAP
        z_tf                = tf.Variable(tf.random_normal(shape=(rand_rest, latent_dim)))
        Y_tf                = tf.placeholder(dtype="float32", shape=(L,L,cam_arr)) #PUT Y HERE AND CHANGE SHAPE

        xG_tf               = G(z_tf)[:,:,:,0]
        #xG_tf               = (xG_tf+1)/2
        yGpred_tf           = forward_map(xG_tf,P_op)



        x_tf  = tf.Variable(tf.random_normal(mean = 0.5, stddev  = 0.01,shape=([1,L,L]),dtype = 'float32'),constraint=lambda t: tf.clip_by_value(t, 0, 1))
        #x_tf = tf.clip_by_value(x_tf,0,1)

        loss1 = tf.reduce_mean( tf.square( Y_tf - yGpred_tf ))
        loss2 = tf.reduce_mean( tf.square( Y_tf - forward_map(x_tf,P_op)))
        loss3 =  tf.reduce_mean(tf.square(x_tf - xG_tf))
        loss4 = tf.image.total_variation(x_tf)

        Loss_tf = 0.0*loss1 + 50.0*loss2 + 0.01*loss3 #+ 0.0000005*loss4
        opt     = optimizer.minimize(Loss_tf, var_list=[z_tf,x_tf])

        # SOLVING LEAST SQUARES
        # SOLVING LEAST SQUARES
        # SOLVING LEAST SQUARES
        sess = K.get_session()
        sess.run(tf.variables_initializer([z_tf,x_tf]))
        Loss = []
        for i in tqdm(range(STEPS)):
            _ , loss = sess.run([opt,Loss_tf], feed_dict={Y_tf:abs_ifft_mul}) #put Y and A here too
            if (i %50) == 0:
                x_hat= sess.run([x_tf])
                x_hat = np.array(x_hat)
                print(compare_ssim(im1,x_hat[0,0,:,:].astype('float64')))
                plt.imshow(x_hat[0,0,:,:],cmap='gray')
                plt.show() 
            Loss.append(np.mean(loss))
        
        scipy.imsave('test'+str(kk+1)+'_subsampled_'+str(j+1)+'.png',x_hat[0,0,:,:])